# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang

***

## Set up and Import Libraries

In [1]:
#install libraries if needed
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [3]:
#set the root path, change the directory into the project folder
os.chdir("/Users/silvia/cffs-label")

In [4]:
#enable reading data in the scrolling window 
pd.set_option("display.max_rows", None, "display.max_columns", None)

***

## Import Preprocessed Datasets

In [5]:
#read Items_List.csv
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.0,ea,1.0,lb,SPICES
1,I-4971,ARTICHOKE 1/4 SALAD CUT TFC,6.0,LG CAN,2.5,Kg,PRODUCE
2,I-4473,AVOCADO (20CT) MX,20.0,CT,1.0,HEAD,PRODUCE
3,I-4973,AVOCADO PULP CHUNKY,12.0,bag,454.0,g,PRODUCE
4,I-4496,BAK CHOY BABY BC,30.0,lb,1.0,lb,PRODUCE


In [7]:
Items.shape

(488, 7)

In [8]:
#read Ingredients_List.csv
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [9]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3388,1.00,L,1.00000,0.3058,P-10496
1,I-4660,2.27,Kg,2.20462,0.6942,P-10496
2,I-4598,1.00,CT,1.00000,0.0013,P-12954
3,I-4679,1.00,BUNCH,1.00000,0.0063,P-18318
4,I-4792,10.00,Kg,2.20462,1.2048,P-18746


In [10]:
Ingredients.shape

(3227, 6)

In [11]:
#read Preps_List.csv
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [12]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-56398,BATCH|Guacamole,2.750,Kg,PREP
1,P-24750,CHOPPED|Cilantro,0.500,Kg,NaN
2,P-41574,COOKED|Black Beans,30.000,Kg,PREP
3,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP
4,P-28258,COOKED|Chow Mein,48.081,Kg,PREP


In [13]:
Preps.shape

(493, 5)

In [14]:
#read Product_List.csv
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [15]:
Products.head()

,ProdId,Description,SalesGroup
0,R-30154,ADD|Crackers,OK - CUSTOM KITCHEN
1,R-56337,ALF|Flatbread|Mediterranean,OK - AL FORNO
2,R-61779,ALF|Flatbread|Mushroom Pesto,OK - AL FORNO
3,R-50590,ALF|Flatbread|OK,OK - AL FORNO
4,R-50494,ALF|Flatbread|Proscuitto,OK - AL FORNO


In [16]:
Products.shape

(321, 3)

In [17]:
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [18]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [19]:
Conversions.shape

(484, 6)

***
## Update Conversion List

In [20]:
Update_Conv = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Conv_UpdateConv.csv"))
Update_Conv.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-1028,0.008333,1.0,CT,120.0,g
1,I-1034,0.008333,1.0,CT,120.0,g
2,I-1035,0.010000,1.0,CT,100.0,g
3,I-10605,0.008850,1.0,CT,113.0,g
4,I-1126,0.006667,1.0,CT,150.0,g


In [21]:
for index, row in Update_Conv.iterrows():
    Id = Update_Conv.loc[index, 'ConversionId']
    Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace = True)

In [22]:
frames = [Conversions, Update_Conv]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [23]:
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0000,XXX,1.0000,L
1,NaN,0.877193,1.0000,1.14L,1.1400,L
2,NaN,0.666667,1.0000,1.5L,1.5000,L
3,NaN,0.571429,1.0000,1.75 L,1.7500,L
4,NaN,0.500000,1.0000,2L,2.0000,L
5,NaN,0.250000,1.0000,4L,4.0000,L
6,NaN,0.083333,1.0000,FOOT,12.0000,INCH
7,NaN,0.062500,1.0000,16L,16.0000,L
8,NaN,0.059172,1.0000,1/2LTR,16.9000,fl oz
9,NaN,0.039370,1.0000,750ML,25.4000,fl oz


In [24]:
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

### Create Unit Converter

In [25]:
#import standard unit conversion information and construct a dataframe
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [26]:
#seperate uoms that converted to 'ml' or 'g'
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [27]:
#construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [28]:
#test the std_converter
#assert std_converter(0.25,'lb') == (113.398, 'g')

In [29]:
#construct a unit converter for specific ingredients
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [30]:
#test the spc_converter
#assert spc_converter('I-1120', 1, 'CT') == (50, 'g')

***
## Items with Non-standard Units

In [31]:
col_names = list(Ingredients.columns.values)
Items_Nonstd = []

for index, row in Ingredients.iterrows():
    Ingre = Ingredients.loc[index,'IngredientId']
    Uom = Ingredients.loc[index,'Uom']
    if Uom not in ['g', 'ml'] and Uom not in liquid_unit + solid_unit and Ingre.startswith('I') and Ingre not in Conversions["ConversionId"].tolist():
        Dict = {}
        Dict.update(dict(row))
        Items_Nonstd.append(Dict)

Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
Items_Nonstd.drop_duplicates(subset=['IngredientId'], inplace=True,)
Items_Nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-8856,6.000,ea,1.0,1.0000,R-64671
1,I-64492,1.000,LOAF,1.0,0.0625,P-26234
2,I-1254,0.500,CT,1.0,0.5000,P-28369
3,I-1273,1.000,LOAF,1.0,0.1351,P-58370
4,I-62225,4.000,CT,1.0,0.4444,P-64456
5,I-62736,1.000,ea,1.0,1.0000,R-28249
8,I-2281,0.076,650G,1.0,1.0000,R-30524
9,I-2669,0.125,PIE,1.0,1.0000,R-30673
13,I-1223,1.000,CT,1.0,1.0000,R-54456
30,I-1252,1.000,CT,1.0,1.0000,R-64997


In [32]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Items_Nonstd.csv")
Items_Nonstd.to_csv(path, index = False, header = True)

***
## Clean Preps Units

In [33]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [34]:
#convert uom into 'g' or 'ml' for each prep using the unit converter
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

In [35]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-56398,BATCH|Guacamole,2.750,Kg,PREP,2750.000000,g
1,P-24750,CHOPPED|Cilantro,0.500,Kg,NaN,500.000000,g
2,P-41574,COOKED|Black Beans,30.000,Kg,PREP,30000.000000,g
3,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP,1200.000000,g
4,P-28258,COOKED|Chow Mein,48.081,Kg,PREP,48081.000000,g
5,P-50795,COOKED|Corn,300.000,g,NaN,300.000000,g
6,P-50497,COOKED|Pasta|Spag|WW,4.000,Kg,PREP,4000.000000,g
7,P-26143,COOKED|Rice|Brown Basmati,4.536,Kg,PREP,4536.000000,g
8,P-34121,COOKED|StirFryChicken,4.000,Kg,PREP,4000.000000,g
9,P-61738,CST|Sauces,60.000,ml,NaN,60.000000,ml


In [36]:
# save cleaned preps list to file
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

### Get Preps with Nonstandard Unit

In [37]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [38]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-33556,COOKED| Fried Fish,1.0,each,NaN,1.0,each
1,P-64456,POP-UP|Coconut|Flan|LM,9.0,PTN,NaN,9.0,PTN
2,P-64513,POP|Salmon|En|Papillote|LM,1.0,PTN,NaN,1.0,PTN
3,P-44585,PREP|Lime|WEDGE,8.0,piece,NaN,8.0,piece
4,P-64944,Sesame|Tuna,1.0,PTN,NaN,1.0,PTN


In [39]:
#filter out preps with nonstandard uom but have information already
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))

col_names = list(Preps_Nonstd.columns.values)
Preps_Nonstd_na = []

for index, row in Preps_Nonstd.iterrows():
    PrepId = Preps_Nonstd.loc[index,'PrepId']
    if PrepId not in Manual_PrepU['PrepId'].values:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd_na.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd_na, columns = col_names)
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-33556,COOKED| Fried Fish,1.0,each,NaN,1.0,each
1,P-64456,POP-UP|Coconut|Flan|LM,9.0,PTN,NaN,9.0,PTN
2,P-64513,POP|Salmon|En|Papillote|LM,1.0,PTN,NaN,1.0,PTN
3,P-44585,PREP|Lime|WEDGE,8.0,piece,NaN,8.0,piece
4,P-64944,Sesame|Tuna,1.0,PTN,NaN,1.0,PTN


In [40]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

***

## New Items

In [41]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [42]:
Items_Assigned.shape

(1921, 8)

### Get the List of New Items

In [43]:
#filter new items by itemID that not in the database and output them in a dataframe
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [44]:
New_Items.insert(1, "CategoryID", '')
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-13422,,BURGER 4OZ NATURAL HALAL,1.0,cs,42.0,CT,MEAT
1,I-63034,,BURGER VEG MALIBU GARDENBURGER,48.0,CT,1.0,CT,FOOD - GROCERY
2,I-64468,,CHICK BREAST CRUNCH BREADED FZ,4.0,Kg,1.0,Kg,POULTRY
3,I-1254,,"CIABATTA BUN 5""X 5"" PLAIN",12.0,CT,1.0,CT,BREAD
4,I-62225,,EGG LRG 15 DOZEN LOOSE,15.0,DOZ,12.0,CT,DAIRY
5,I-62736,,HAMBURGER BUN WW VEGAN 100gr,1.0,ea,1.0,ea,BREAD
6,I-64492,,LOAF GARLIC BREAD,1.0,LOAF,1.0,LOAF,BREAD
7,I-3356,,MILK CONDENSED SWEET,24.0,SM CAN,300.0,ml,FOOD - GROCERY
8,I-1223,,"PANINI SUB ITALIAN - 7""",12.0,CT,1.0,CT,BREAD
9,I-53707,,PAPRIKA BULK,5.0,lb,1.0,lb,SPICES


In [45]:
New_Items.shape

(16, 8)

In [46]:
# store the list of new items into .csv file
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

***
## Data Summary

In [47]:
datasum = pd.DataFrame([New_Items.shape, Preps_Nonstd.shape, Items_Nonstd.shape],
                       columns = ['count', 'columns'], 
                       index = ['New_Items', 'Preps_Nonstd', 'Items_Nonstd'])
datasum

,count,columns
New_Items,16,8
Preps_Nonstd,5,7
Items_Nonstd,10,6
